<a href="https://colab.research.google.com/github/Kamalalayagummadi/Handwriting-recognition-Using-CNN-BiLSTM-CTC/blob/main/Handwritten_Character_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Character Recognition

**Mid Project - 2**

**Import Necessary Libraries**

In [71]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Bidirectional, LSTM, Lambda
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.backend import ctc_batch_cost, ctc_decode

**1. Read the parser.txt file containing the image id and the respective word for that image and take the first 10000 instances for training and testing of the model**





In [72]:

# Define the path to your parser.txt file
parser_file_path = '/content/drive/MyDrive/parser/parser.txt'


In [73]:
# Initialize an empty list to store valid data
data_list = []

# Read the parser.txt file line by line
with open(parser_file_path, 'r') as file:
  for line_number, line in enumerate(file, start=1):
    try:
# Split the line into Image_ID and Word
      image_id, word = line.strip().split(' ', 1)
      data_list.append({'Image_ID': image_id, 'Word': word})
    except ValueError:
        print(f"Skipping line {line_number} due to unexpected structure.")

In [ ]:
print("Sample entries in data_list:")
print(data_list[:5])

Sample entries in data_list:
[{'Image_ID': 'a01-000u-00-00', 'Word': 'ok 154 408 768 27 51 AT A'}, {'Image_ID': 'a01-000u-00-01', 'Word': 'ok 154 507 766 213 48 NN MOVE'}, {'Image_ID': 'a01-000u-00-02', 'Word': 'ok 154 796 764 70 50 TO to'}, {'Image_ID': 'a01-000u-00-03', 'Word': 'ok 154 919 757 166 78 VB stop'}, {'Image_ID': 'a01-000u-00-04', 'Word': 'ok 154 1185 754 126 61 NPT Mr.'}]


In [ ]:
# Create a DataFrame from the list of valid data
df = pd.DataFrame(data_list)

In [ ]:
df.head()

,Image_ID,Word
0,a01-000u-00-00,ok 154 408 768 27 51 AT A
1,a01-000u-00-01,ok 154 507 766 213 48 NN MOVE
2,a01-000u-00-02,ok 154 796 764 70 50 TO to
3,a01-000u-00-03,ok 154 919 757 166 78 VB stop
4,a01-000u-00-04,ok 154 1185 754 126 61 NPT Mr.


In [ ]:
# Take the first 10000 instances for training and testing
df_subset = df.head(10000)

In [ ]:
df_subset.head()

,Image_ID,Word
0,a01-000u-00-00,ok 154 408 768 27 51 AT A
1,a01-000u-00-01,ok 154 507 766 213 48 NN MOVE
2,a01-000u-00-02,ok 154 796 764 70 50 TO to
3,a01-000u-00-03,ok 154 919 757 166 78 VB stop
4,a01-000u-00-04,ok 154 1185 754 126 61 NPT Mr.


**6. Split your dataset for training and testing**

In [ ]:
# Separate into training and testing sets (e.g., 80% for training, 20% for testing)
train_size = int(0.8 * len(df_subset))
train_data, test_data = df_subset[:train_size], df_subset[train_size:]

In [ ]:
# Print a few rows to verify the data
print("Training Data:")
print(train_data.head())

print("\nTesting Data:")
print(test_data.head())

Training Data:
         Image_ID                            Word
0  a01-000u-00-00       ok 154 408 768 27 51 AT A
1  a01-000u-00-01   ok 154 507 766 213 48 NN MOVE
2  a01-000u-00-02      ok 154 796 764 70 50 TO to
3  a01-000u-00-03   ok 154 919 757 166 78 VB stop
4  a01-000u-00-04  ok 154 1185 754 126 61 NPT Mr.

Testing Data:
           Image_ID                              Word
8000  a03-050-02-01  ok 162 588 1104 263 58 VB regard
8001  a03-050-02-02   ok 162 938 1096 119 47 PP3O him
8002  a03-050-02-03       ok 162 1162 1137 28 19 *' "
8003  a03-050-02-04      ok 162 1224 1118 87 31 IN as
8004  a03-050-02-05  ok 162 1370 1115 185 28 DTI some


**2. Images can be of different shape thus resize all your images to have the same shape (forexample = (128,32))**

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the directory containing images
images_directory = '/content/drive/MyDrive/words'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! ls '/content/drive/MyDrive/words'

a01  a05  b03  c01  c06  d05  e02  f01	f07  g04  h01  h06  j06  k03  l03  m02	n01  n06  p06
a02  a06  b04  c02  d01  d06  e04  f02	g01  g05  h02  h07  j07  k04  l04  m03	n02  p01  r02
a03  b01  b05  c03  d03  d07  e06  f03	g02  g06  h04  j01  k01  k07  l07  m04	n03  p02  r03
a04  b02  b06  c04  d04  e01  e07  f04	g03  g07  h05  j04  k02  l01  m01  m06	n04  p03  r06


In [ ]:
# Check the number of files in the original images_directory
print(f"Number of Files in Original Images Directory: {len(os.listdir(images_directory))}")

Number of Files in Original Images Directory: 77


In [ ]:
from sklearn.model_selection import train_test_split

# List of filenames in the image directory
filenames = os.listdir(images_directory)

# Split the data into training and temporary sets
train_filenames, temp_filenames = train_test_split(filenames, test_size=0.4, random_state=42)

# Split the temporary set into testing and validation sets
test_filenames, val_filenames = train_test_split(temp_filenames, test_size=0.5, random_state=42)

# Print the number of samples in each set
print("Number of training samples:", len(train_filenames))
print("Number of testing samples:", len(test_filenames))
print("Number of validation samples:", len(val_filenames))


Number of training samples: 46
Number of testing samples: 15
Number of validation samples: 16


In [ ]:
from PIL import Image

# Define the target shape
target_shape = (128, 32)

# Create a list to store resized images
resized_images = []

# Iterate through each image in the directory
for filename in os.listdir(images_directory):
    if filename.endswith(".png"):  # Adjust the file extension as needed
        # Construct the full path to the image
        image_path = os.path.join(images_directory, filename)

        # Open and resize the image
        img = Image.open(image_path)
        img = img.resize(target_shape)

        # Convert the image to a NumPy array
        img_array = np.array(img)

        # Append the resized image to the list
        resized_images.append(img_array)

# Convert the list of resized images to a NumPy array
resized_images = np.array(resized_images)

In [ ]:
# Print the shape of the resized images array
print(resized_images.shape)

(0,)


**3. Currently, the pixel values are between 0 to 255, normalize the images so that the pixel
values are in range 0 to 1**

In [ ]:
# Normalize pixel values to be in the range [0, 1]
normalized_images = resized_images / 255.0

**4. Create a list of all characters and use the character’s index to encode the actual words
into digits**

In [ ]:
# Create a list of all unique characters in the words
all_characters = sorted(set(' '.join(df['Word'])))
num_classes = len(all_characters) + 1  # Add 1 for the blank character

# Create a dictionary mapping characters to their respective indices
char_to_index = {char: idx for idx, char in enumerate(all_characters)}

# Create a dictionary mapping indices to their respective characters
index_to_char = {idx: char for idx, char in enumerate(all_characters)}

# Function to encode a word into a sequence of digits
def encode_word(word):
    return [char_to_index[char] for char in word]

# Apply the encoding to the 'Word' column in the DataFrame
train_data['Encoded_Word'] = train_data['Word'].apply(encode_word)
test_data['Encoded_Word'] = test_data['Word'].apply(encode_word)


<ipython-input-23-32cd7c584fb8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Encoded_Word'] = train_data['Word'].apply(encode_word)
<ipython-input-23-32cd7c584fb8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Encoded_Word'] = test_data['Word'].apply(encode_word)


In [ ]:
# Example output for the 'Encoded_Word' column in the training set
print("Encoded Words in the Training Set:")
print(train_data['Encoded_Word'].head())

# Example output for the 'Encoded_Word' column in the testing set
print("\nEncoded Words in the Testing Set:")
print(test_data['Encoded_Word'].head())


Encoded Words in the Training Set:
0    [68, 64, 0, 16, 20, 19, 0, 19, 15, 23, 0, 22, ...
1    [68, 64, 0, 16, 20, 19, 0, 20, 15, 22, 0, 22, ...
2    [68, 64, 0, 16, 20, 19, 0, 22, 24, 21, 0, 22, ...
3    [68, 64, 0, 16, 20, 19, 0, 24, 16, 24, 0, 22, ...
4    [68, 64, 0, 16, 20, 19, 0, 16, 16, 23, 20, 0, ...
Name: Encoded_Word, dtype: object

Encoded Words in the Testing Set:
8000    [68, 64, 0, 16, 21, 17, 0, 20, 23, 23, 0, 16, ...
8001    [68, 64, 0, 16, 21, 17, 0, 24, 18, 23, 0, 16, ...
8002    [68, 64, 0, 16, 21, 17, 0, 16, 16, 21, 17, 0, ...
8003    [68, 64, 0, 16, 21, 17, 0, 16, 17, 17, 19, 0, ...
8004    [68, 64, 0, 16, 21, 17, 0, 16, 18, 22, 15, 0, ...
Name: Encoded_Word, dtype: object


**5. Pad all the words to have a similar length**

In [ ]:
# Find the maximum length of words in the training set
max_word_length = max(train_data['Encoded_Word'].apply(len))

# Pad all words to have the same length
train_padded_words = pad_sequences(train_data['Encoded_Word'], maxlen=max_word_length, padding='post')
test_padded_words = pad_sequences(test_data['Encoded_Word'], maxlen=max_word_length, padding='post')


In [ ]:
# Print the first few padded sequences in the training set
print("Padded sequences in the training set:")
print(train_padded_words[:5])

# Print the first few padded sequences in the testing set
print("\nPadded sequences in the testing set:")
print(test_padded_words[:5])

Padded sequences in the training set:
[[68 64  0 16 20 19  0 19 15 23  0 22 21 23  0 17 22  0 20 16  0 28 47  0
  28  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [68 64  0 16 20 19  0 20 15 22  0 22 21 21  0 17 16 18  0 19 23  0 41 41
   0 40 42 49 32  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [68 64  0 16 20 19  0 22 24 21  0 22 21 19  0 22 15  0 20 15  0 47 42  0
  73 68  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [68 64  0 16 20 19  0 24 16 24  0 22 20 22  0 16 21 21  0 22 23  0 49 29
   0 72 73 68 69  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [68 64  0 16 20 19  0 16 16 23 20  0 22 20 19  0 16 17 21  0 21 16  0 41
  43 47  0 40 71 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]

Padded sequences in the testing set:
[[68 64  0 16 21 17  0 20 23 23  0 16 16 15 19  0 17 21 18  0 20 23  0 49
  29  0 71 58 60 54 71 57  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [68 64  0 16 21 17  0 24 18 23  0 16 15 24 21  0 16 16 24  0 19 

**7. Create a model for training:**

a) Add several CNN layers to extract the sequence of features

b) Add Bi-LSTM layers to propagate through the sequence

c) Add a dense layer (output layer) with total number of neurons as (total number ofcharacters + 1) and the activation as softmax

In [ ]:
from keras.layers import Reshape

# Define the model
input_shape = target_shape + (3,)  # Assuming images are in color (RGB)

inputs = Input(shape=input_shape, name='input_image')
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

# Flatten the feature map
x = Flatten()(x)

# Reshape to 3D
x = Reshape((-1, 1))(x)

# Bidirectional LSTM layers
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)

# Output layer
outputs = Dense(num_classes, activation='softmax', name='output')(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Print the summary of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 128, 32, 3)]      0         
                                                                 
 conv2d (Conv2D)             (None, 128, 32, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 8, 64)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 16384)             0     

**8. The output sequence from the output layer will be fed to the CTC layer.**

In [ ]:
# CTC loss layer
input_length = np.ones((len(train_padded_words), 1)) * max_word_length
label_length = np.array([len(seq) for seq in train_padded_words]).reshape(-1, 1)

# Define the CTC loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Connect the CTC loss layer to the model
ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, train_padded_words, input_length, label_length])

# Create the final model for training
model_train = Model(inputs=inputs, outputs=ctc_loss)

# Compile the final model
optimizer = Adam(learning_rate=0.001)
model_train.compile(optimizer=optimizer, loss={'ctc': lambda y_true, y_pred: y_pred}, metrics=['accuracy'])


In [ ]:
# Print the summary of the model
model_train.summary()

# Print additional information
print(f"Number of training examples: {len(train_padded_words)}")
print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 128, 32, 3)]      0         
                                                                 
 conv2d (Conv2D)             (None, 128, 32, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 8, 64)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 16384)             0   

**9. Predict output using your model (do not use the last loss layer) on validation images, use ctc_decode to decode your output and then print the actual words using the indexes
from your characters list.**

In [ ]:
# Load and preprocess validation images
val_images = []

for filename in val_filenames:
    image_path = os.path.join(images_directory, filename)

    # Skip directories
    if os.path.isdir(image_path):
        continue

    try:
        img = Image.open(image_path)
        img = img.resize(target_shape)
        img_array = np.array(img) / 255.0  # Normalize pixel values
        val_images.append(img_array)
    except Exception as e:
        print(f"Error processing image {filename}: {e}")

# Convert val_images to a numpy array
val_images = np.array(val_images)

# Print the filenames being processed
print("Filenames being processed:")
print(val_filenames)

# Print the shape of the val_images array
print("Shape of val_images array:", val_images.shape)

# Ensure that val_images is not empty before predicting
if val_images.shape[0] == 0:
    print("No valid images found in the validation set.")
else:
    # Predict output using the model (excluding the last loss layer)
    predictions = model.predict(val_images)

    # Use ctc_decode to decode the output
    decoded, _ = K.ctc_decode(predictions, input_length=np.ones(predictions.shape[0]) * predictions.shape[1])

    # Convert the decoded indices to actual words
    decoded_words = []
    for i, sequence in enumerate(decoded):
        decoded_word = ''.join([index_to_char[idx] for idx in sequence if idx != -1])
        decoded_words.append(decoded_word)

    # Print actual words and their decoded counterparts
    actual_words = test_data.loc[test_data['Image_ID'].isin(val_filenames), 'Word'].tolist()

    for actual, predicted in zip(actual_words, decoded_words):
        print(f"Actual: {actual}, Predicted: {predicted}")


Filenames being processed:
['r03', 'h05', 'm01', 'r06', 'g03', 'j07', 'f07', 'c03', 'n02', 'p03', 'f01', 'n06', 'g02', 'm04', 'a04', 'h02']
Shape of val_images array: (0,)
No valid images found in the validation set.
